### Image retrieval function to be used by Grafana

In [8]:
import nuclio

In [9]:
%run set_env.ipynb

In [10]:
# nuclio: start-code

In [46]:
import requests
import json
import base64
import os
import time

In [47]:
headers = {
            "Content-Type": "application/json",
            "X-v3io-session-key" : os.getenv("V3IO_ACCESS_KEY")
          }

In [37]:
def seek_location(context,
                  shardId: int = 0,
                  container: str = '',
                  stream: str = "",
                  url = '',
                 secondsLapsed = 1):
    payload = {
            "ShardId": shardId,
            "Type": "TIME",
        "TimestampSec":int(time.time())-secondsLapsed,
        "TimestampNSec" : 0
          }
    headers["X-v3io-function"] = "Seek"
    r = requests.post(url, headers=headers,json=payload)
    return json.loads(r.text)['Location']

In [38]:
def get_image( context,
              url,
              shardId,
              container,
              stream,
              secondsLapsed):
    location = seek_location(context,shardId=shardId,
                             container=container,
                             stream=stream,
                             url=url,
                            secondsLapsed = secondsLapsed)
    img_payload = {
            "ShardId":    shardId,
            "Location":   location,
        "Limit":1
    }
    headers["X-v3io-function"] = "GetRecords"
    r = requests.post(url, headers=headers,json=img_payload)
    
    response=json.loads(r.text)
    
    return response

In [53]:
def handler(context,event):
    params = event.fields
    shardId=int(params['shardId'])
    container=params['container']
    stream=params['stream']
    
    # Fire up the Kafka Consumer
    url = "http://v3io-webapi:8081/%s/%s/"% (container,stream)
    headers = { "Content-Type": "application/json",
            "X-v3io-session-key" : os.getenv("V3IO_ACCESS_KEY")
    }

    secondsLapsed = 1
    while True:
          response = get_image(context,url,shardId,container,stream,secondsLapsed )
          data = False  
          for record in response['Records']:
                data = base64.b64decode(record['Data']) 
          if data:
            break
          else:
              secondsLapsed += 1
    
    #img = np.asarray(bytearray(data), dtype="uint8")
    #img = cv2.imdecode(img,cv2.COLOR_BGR2RGBA)
    #cv2.imshow('Faces',img)
    #return context.Response(body=data,headers=None,content_type='image/jpeg',status_code=201)
    return data

In [54]:
# nuclio: end-code

In [55]:
import os

In [56]:
# nuclio: ignore
# converts the notebook code to deployable function with configurations
from mlrun import code_to_function, mount_v3io, mlconf
import os

fn = code_to_function('latest-image-retrieve', kind='nuclio')

In [57]:
# set the API/trigger, attach the home dir to the function
fn.with_http(workers=1).apply(mount_v3io())
#fn.spec.build.base_image = 'mlrun/mlrun'
fn.spec.base_spec['spec']['build']['baseImage'] = 'python:3.6-jessie'
fn.spec.build.commands = ['pip install opencv-python==4.2.0.34 v3io_frames requests']
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1
fn.set_env('V3IO_ACCESS_KEY',os.getenv('V3IO_ACCESS_KEY'))
fn.set_env('IGZ_CONTAINER',IGZ_CONTAINER)
fn.set_env('RAW_VIDEO_STREAM',RAW_VIDEO_STREAM)
fn.set_env('TAGGED_VIDEO_STREAM',TAGGED_VIDEO_STREAM)
fn.set_env('CAMERA_LIST_TBL',CAMERA_LIST_TBL)
fn.apply(mount_v3io())

In [58]:
#print(fn.to_yaml())

In [59]:
# nuclio: ignore
addr = fn.deploy(project='objectrecognition')

> 2020-08-18 16:46:41,526 [info] deploy started
[nuclio] 2020-08-18 16:46:48,744 (info) Build complete
[nuclio] 2020-08-18 16:46:54,827 (info) Function deploy complete
[nuclio] 2020-08-18 16:46:54,836 done updating objectrecognition-latest-image-retrieve, function address: 3.129.97.78:31298
